In [ ]:
import os
import shutil
import time
import pandas as pd
from tqdm.notebook import tqdm      #tqdm(filelist, total = len(filelist, position=0, leave=Ture)
import re
from pathlib import Path
import traceback
from os.path import join

### 채무자키_대상자명 넣기_선정리

In [ ]:
#[시작]##########################################################
# -1...로 넘버링 된거 _(1)..로 변환
path = r'D:\1.다운로드\부산 공유폴더\3. (9008) 더키움자산관리대부_헬프\3. 채권양도통지서\3. 부산발송양통'
os.chdir(path)
file_list = os.listdir(path)
index = 0
p1 = re.compile('[-][\d][.]')

for f in file_list :
    temp = p1.search(f)
    if temp :
        
        new_name = os.path.splitext(f)[0][:-2] + "_("+ temp.group()[1] +")" +os.path.splitext(f)[1]
        os.rename(f, new_name)
        
        index += 1

print(index, "개 완료")
os.chdir('c:/')
#[끝]##########################################################

In [ ]:
path = r'C:\Users\SL\Downloads\부산 공유폴더\1.한울가람_헬프\2. 원인서류(대출신청서)\한울 서고2 원장스캔'
f_d = r"C:\Users\SL\Downloads\부산 공유폴더\1.한울가람_헬프\2. 원인서류(대출신청서)\한울 서고2" #폴더내에서 이름 변경시 동일 이름이 존재하게 되면 까다로우니 새 폴더를 만들어서 처리하자. 새폴더

In [ ]:
#[시작]##########################################################
##맨앞 일련번호와 대상자명 사이에 _ 없는 거 추가 '숫자4개+글자' 또는 '숫자4개-1개+글자'
os.chdir(path)
file_list = os.listdir(path)
_ = '_'
index = 0

for f in file_list:
    # 새 파일명 만들기
    #fullname = Path(Path.cwd() / f)  # 경로+파일명+확장자
    #splitname = fullname.stem.split('_')  # _로 쪼개기(배열)
    #existing_f = f  # f를 살려야 하니 새 변수에
    new_name = f
    
    a = re.match('\d\d\d\d-[\d]{1,}', f)
    b = re.match('\d\d\d\d', f)
    
    if a != None : 
        new_name = a.group() + _ + f[a.span()[1]:]
    elif b != None : 
        new_name = b.group() + _ + f[b.span()[1]:]

    new_name = re.sub('_{2,}', _, new_name)
   
    
    temp = Path(f_d + "/"+ new_name)
    i = 1
    while os.path.exists(f_d+"/"+new_name): # os.path는 무조건 경로를 매개변수로..작업디렉토리가 아니므로 풀경로, new_name이 매개변수에 포함되어야 함!
        new_name = temp.stem + _ +"("+str(i)+")"+temp.suffix
        i += 1
                    
    f_d_final = f_d + "/" + new_name
    os.rename(f, f_d_final)
    index += 1

print(index, "개의 파일 이름 변경 완료")
os.chdir('C:/') # 작업디렉토리 옮겨줘야 폴더 삭제 가능
#[끝]##########################################################

In [ ]:
#[시작]###########################################################
# 채무자키 있는 애들 추려서 옮기기
path = r'C:\Users\SL\Downloads\부산 공유폴더\1.한울가람_헬프\2. 원인서류(대출신청서)\한울 서고2'
f_d = r'D:\부산 정리\한울가람대부㈜\temp'
os.chdir(path)
file_list = os.listdir(path)
index = 0
p1 = re.compile("\d\d\d\d_\d\d\d\d")

for f in file_list:
    if re.match(p1, f) :
        new_name = f[:4]+f[5:]

        temp = Path(f_d + "/"+ new_name)
        i = 1
        while os.path.exists(f_d+"/"+new_name): # os.path는 무조건 경로를 매개변수로..작업디렉토리가 아니므로 풀경로, new_name이 매개변수에 포함되어야 함!
            new_name = temp.stem + _ +"("+str(i)+")"+temp.suffix
            i += 1
                        
        f_d_final = f_d + "/" + new_name
        shutil.move(f, f_d_final)
        index += 1

print("작업 완료 개수", index)
#[끝]###########################################################

In [ ]:
#[시작]###########################################################
#(주)는 ㈜로 바꾸고 넘버링 제외한 괄호 다 지우기
path = r'C:\Users\SL\Downloads\부산 공유폴더\1.한울가람_헬프\2. 원인서류(대출신청서)\한울 서고2'
_ = "_"
os.chdir(path)
file_list = os.listdir(path)

p1 = re.compile('\(주\)|\(주')
p2 = re.compile('\([\D]+\)') 
for f in file_list :
        
    changed = False
    #new_name = "" #이거 안 해서 (주)그냥 다 빠져버렸네. ;; 어차피 지우고 검색할 거니까.. 

    if p1.search(f) :
        new_name = p1.sub("㈜", f)
        changed = True

    if p2.search(f) : #괄호 사이에 공백 있었으면 f1에 다 안 담기므로
        new_name = p2.sub("", f)
        changed = True
    
    if changed :
        os.rename(f, new_name)

os.chdir('C:/')

### 채무자키_대상자명 넣기

In [ ]:
## 참조 df 불러오기 ~~~~ trim()함수로 앞뒤공백만 제거 ~~~~
## 참조할 col_name = '채무자키', '매각사구분', 성명', '주민번호', '매각사', '보증인성명'
                ###################
sell_company = "대산대부"
                ###################
df_c = pd.read_excel(r'C:\Users\SL\Desktop\채무자조회.xlsx')
df_c = df_c[df_c.매각사==sell_company]   # df_c.loc[조건,:] 이렇게 안 해도 성능차이 없이 되네!!
df_c.head()

In [ ]:
path = r'D:\1.다운로드\부산 공유폴더\5. (9006) 대산대부(태산)_헬프\1. 채권양도통지서\대산대부-차주'
f_d = r'D:\부산 정리\대산대부'
docu_kind = '양도통지서'

#etc = ["재도", "확정증명원", "보증인"] #키워드 있는 경우
etc_str = "" #_보증인" #언더바 붙여야 돼!!!! 키워드 없는 경우, 폴더도 없다면(주채무자인경우) ""로 처리

In [ ]:
os.chdir(path)
file_list = os.listdir(path)
_ = '_'
index = 0
###### 빈리스트로 나올거라 생각하지만 혹시나
admin_duple = []
debtor_duple = []
no_search = []
############################################
f = ""
p1 = re.compile(r'([\d]{3})-\d\d-\d\d\d\d\d')
p2 = re.compile(r'([\d]{6})-([\d]{6})')
# 앞에 4자리가 같은 것끼리 세트로 묶고 그중에 관리자키 있는 거로 채무자키 찾아서 ~

# try :
    
for f in file_list : 
    if f != "Thumbs.db" :
        fullname = Path(Path.cwd() / f)  # 경로+파일명+확장자
        splitname = fullname.stem.split('_')  # 확장자 제외하고 _로 쪼개기(배열)
        
        changed = False

        if (re.search('[0-9]{10}', f) != None) & (re.search('[0-9]{11,}', f) == None) : #관리자키 있으면
            admin = re.search('[0-9]{10}', f).group()
            df_ = df_c[df_c.관리자기타==admin]     # 참조df에서 관리자키 일치하는 거 찾아서
            if len(df_.index)==1 :                # 일치하는 게 하나만 나왔을 때
                splitname[0] = str(df_.채무자키.iloc[0])     # 맨 앞자리를 채무자키로 대체
                splitname[1] = str(df_.성명.iloc[0])    #인덱싱 결과는 시리즈이기 때문에 .iloc[0]붙여야
                changed = True
            elif len(df_.index)>1 :
                admin_duple.append(f)           # 일치값이 여러개나오면 별도 리스트로
            else :
                no_search.append(f)             # 일치값이 없을 때
        

        elif (p1.search(f) != None) | (p2.search(f) != None) :   # 관리자키 없으면 사업자번호 / 주민번호로 찾자
            if p1.search(f) != None :
                number = p1.search(f).group()
            else :
                number = p2.search(f).group()

            df_ = df_c[df_c.주민번호==number]
            if len(df_.index) ==1:
                splitname[0] = str(df_.채무자키.iloc[0])
                splitname[1] = str(df_.성명.iloc[0])
                changed = True
            elif len(df_.index) >1 :
                admin_duple.append(f)           
            else :
                no_search.append(f)
        
        else : #관리자키도 주민번호도 없으면 이름으로
            name = splitname[1]
            df_ = df_c[df_c.성명.str.contains(name)]
            if len(df_) ==1 :
                splitname[0] = str(df_.채무자키.iloc[0])
                splitname[1] = str(df_.성명.iloc[0])
                changed = True
            elif len(df_.index) >1 :
                admin_duple.append(f)           
            else :
                no_search.append(f)



        # 파일명 바꾸고 이동하기
        if changed : #채무자키 정상 입력 된 경우
            new_name = splitname[0]+_+splitname[1]+_+docu_kind+fullname.suffix

            temp = Path(f_d + "/"+ new_name)
            i = 1
            
            while os.path.exists(f_d+"/"+new_name): # os.path는 무조건 경로를 매개변수로..작업디렉토리가 아니므로 풀경로, new_name이 매개변수에 포함되어야 함!
                new_name = temp.stem + _ +"("+str(i)+")"+temp.suffix
                i += 1
                            
            f_d_final = f_d + "/" + new_name
            shutil.move(f, f_d_final)
            index += 1

# except Exception as e:
#     print(f)
#     print(e)
#     print(index,"개 파일 이동 완료")
#     print("중복키", len(admin_duple))
#     print("키없음", len(no_search))

print("완료")
print(index,"개 파일 이동 완료")
print("중복키", len(admin_duple))
print("키없음", len(no_search))
os.chdir("c:/")


#1)관리자키 있으면 관리자키로 검색 -> 채무자키 하나 검색됐으면 0번 자리를 채무자키로 대신 // 하나 이상이면? 일단 관리자키 리스트 만들어 저장
#2)관리자키 없으면 대상자명 검색 -> 채무자키 하나 검색됐으면 0번 자리를 채무자키로 대신  // 하나 이상이면? 양이 얼마나 되는지 보게 일단 대상자명 리스트 만들어 저장
#3)일치하는 값이 둘다 없다면 -> 참조없음 리스트 만들어 저장

In [ ]:
###########[시작]관리자번호만 있는 경우################
os.chdir(path)
file_list = os.listdir(path)
_ = '_'
index = 0
###### 빈리스트로 나올거라 생각하지만 혹시나
admin_duple = []
debtor_duple = []
no_search = []
############################################
f = ""
p1 = re.compile(r'([\d]{3})-\d\d-\d\d\d\d\d')
p2 = re.compile(r'([\d]{6})-([\d]{6})')
# 앞에 4자리가 같은 것끼리 세트로 묶고 그중에 관리자키 있는 거로 채무자키 찾아서 ~

# try :
    
for f in file_list : 
    if f != "Thumbs.db" :
        fullname = Path(Path.cwd() / f)  # 경로+파일명+확장자
        splitname = [None]*2
        changed = False

        if (re.search('[0-9]{10}', f) != None) & (re.search('[0-9]{11,}', f) == None) : #관리자키 있으면
            admin = re.search('[0-9]{10}', f).group()
            df_ = df_c[df_c.관리자기타==admin]     # 참조df에서 관리자키 일치하는 거 찾아서
            if len(df_.index)==1 :                # 일치하는 게 하나만 나왔을 때
                splitname[0] = str(df_.채무자키.iloc[0])     # 맨 앞자리를 채무자키로 대체
                
                ### 주채무자 vs 보증인
                # 1)주채무자 
                splitname[1] = str(df_.성명.iloc[0])    #인덱싱 결과는 시리지이기 때문에 .iloc[0]붙여야
                 
                # 2)보증인 폴더
                #splitname[1] = str(df_.보증인성명.iloc[0])
                
                # 3)보증인 키워드 & 추가 기타정보 검색해야 할 때
                # for e in etc :
                #     if re.search(e, f):
                #         etc_str += ( _ + e )
                #         splitname[1] = str(df_.보증인성명.iloc[0])
                                                
                changed = True
                
            elif len(df_.index)>1 :
                admin_duple.append(f)           # 일치값이 여러개나오면 별도 리스트로
            else :
                no_search.append(f)             # 일치값이 없을 때


        # 파일명 바꾸고 이동하기 - temp.stem은 넘버링이 붙지 않은 상태이므로 _(1)_(1) 이런 문제가 없다.
        if changed : #채무자키 정상 입력 된 경우
            new_name = splitname[0]+_+splitname[1]+_+docu_kind+ etc_str +fullname.suffix

            temp = Path(f_d + "/"+ new_name)
            i = 1
            
            while os.path.exists(f_d+"/"+new_name): # os.path는 무조건 경로를 매개변수로..작업디렉토리가 아니므로 풀경로, new_name이 매개변수에 포함되어야 함!
                new_name = temp.stem + _ +"("+str(i)+")"+temp.suffix
                i += 1
                            
            f_d_final = f_d + "/" + new_name
            shutil.move(f, f_d_final)
            index += 1

# except Exception as e:
#     print(f)
#     print(e)
#     print(index,"개 파일 이동 완료")
#     print("중복키", len(admin_duple))
#     print("키없음", len(no_search))

print("완료")
print(index,"개 파일 이동 완료")
print("중복키", len(admin_duple))
print("키없음", len(no_search))
os.chdir("c:/")

###########[끝]관리자번호만 있는 경우################

In [ ]:
path = r'D:\부산 정리\KB국민카드'
os.chdir(path)
file_list = os.listdir(path)
p1 = re.compile('양도통지서1차')
for f in file_list :
    if p1.search(f) :
        new_name = p1.sub("양도통지서_1차", f)
        os.rename(f, path + "/" + new_name)
    


In [ ]:
print("중복키", len(admin_duple))
print(admin_duple)
print("키없음", len(no_search))
print(no_search)

In [ ]:
splitname = ['2458', '포스기술㈜']
f = ["㈜123456㈜78901㈜", "포스기술(주)"]
df = pd.DataFrame(f, columns= ['성명'])
splitname[1] = re.sub('㈜',"",splitname[1]) #㈜있으면 제거
print(splitname[1])
df_debtor = df[df.성명.str.contains(splitname[1])]       # 이름으로 검색
df_debtor

#f = re.sub('㈜',"",f)


In [ ]:
#[시작]###########################################################
#후처리 : 한울가람 앞에 넣고, 원인서류 뒤에 넣고

path = r'C:\Users\SL\Downloads\부산 공유폴더\1.한울가람_헬프\2. 원인서류(대출신청서)\한울 서고2'
kind1 = '한울가람_'
kind2 = '_원인서류'

file_list = os.listdir(path)
f = ""
try :
    for f in file_list :
        if f != "Thumbs.db" :
            new_name = kind1 + os.path.splitext(f)[0] + kind2 + os.path.splitext(f)[1]
            os.rename(path + "/" + f, path + "/" + new_name)

except Exception as e:
    print(e)
    print(f)
#[끝]###########################################################